In [1]:
import numpy as np
import itertools
import time
import datetime
import torchvision.transforms as transforms
from torchvision.utils import save_image, make_grid
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch
from matplotlib.pyplot import figure
from IPython.display import clear_output
from PIL import Image
import matplotlib.image as mpimg

from utils import *
from cyclegan import *

# Check if CUDA is available
cuda = torch.cuda.is_available()
print("Using CUDA" if cuda else "Not using CUDA")

# Set Tensor type based on CUDA availability
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

# Define hyperparameters
class Hyperparameters:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

hp = Hyperparameters(
    epoch=0,
    n_epochs=200,
    dataset_train_mode="train",
    dataset_test_mode="test",
    batch_size=4,
    lr=0.0002,
    decay_start_epoch=100,
    b1=0.5,
    b2=0.999,
    n_cpu=8,
    img_size=128,
    channels=3,
    n_critic=5,
    sample_interval=100,
    num_residual_blocks=99,
    lambda_cyc=10.0,
    lambda_id=5.0,
)

# Root path for datasets
root_path = "/content/drive/MyDrive/All_Datasets/summer2winter_yosemite"

# Define image visualization methods
def show_img(img, size=10):
    img = img / 2 + 0.5
    npimg = img.numpy()
    plt.figure(figsize=(size, size))
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

def to_img(x):
    x = x.view(x.size(0) * 2, hp.channels, hp.img_size, hp.img_size)
    return x

def plot_output(path, x, y):
    img = mpimg.imread(path)
    plt.figure(figsize=(x, y))
    plt.imshow(img)
    plt.show()

# Define image transforms
transforms_ = [
    transforms.Resize((hp.img_size, hp.img_size), Image.BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
]

# Load datasets
train_dataloader = DataLoader(
    ImageDataset(root_path, mode=hp.dataset_train_mode, transforms_=transforms_),
    batch_size=hp.batch_size,
    shuffle=True,
    num_workers=1,
)
val_dataloader = DataLoader(
    ImageDataset(root_path, mode=hp.dataset_test_mode, transforms_=transforms_),
    batch_size=16,
    shuffle=True,
    num_workers=1,
)

# Function to save image samples
def save_img_samples(batches_done):
    print("batches_done ", batches_done)
    imgs = next(iter(val_dataloader))

    Gen_AB.eval()
    Gen_BA.eval()

    real_A = imgs["A"].type(Tensor)
    fake_B = Gen_AB(real_A)
    real_B = imgs["B"].type(Tensor)
    fake_A = Gen_BA(real_B)

    real_A = make_grid(real_A, nrow=16, normalize=True)
    real_B = make_grid(real_B, nrow=16, normalize=True)
    fake_A = make_grid(fake_A, nrow=16, normalize=True)
    fake_B = make_grid(fake_B, nrow=16, normalize=True)

    image_grid = torch.cat((real_A, fake_B, real_B, fake_A), 1)
    path = root_path + "/%s.png" % (batches_done)
    save_image(image_grid, path, normalize=False)
    return path

# Loss functions
criterion_GAN = torch.nn.MSELoss()
criterion_cycle = torch.nn.L1Loss()
criterion_identity = torch.nn.L1Loss()

input_shape = (hp.channels, hp.img_size, hp.img_size)

# Initialize generator and discriminator
Gen_AB = GeneratorResNet(input_shape, hp.num_residual_blocks)
Gen_BA = GeneratorResNet(input_shape, hp.num_residual_blocks)
Disc_A = Discriminator(input_shape)
Disc_B = Discriminator(input_shape)

if cuda:
    Gen_AB = Gen_AB.cuda()
    Gen_BA = Gen_BA.cuda()
    Disc_A = Disc_A.cuda()
    Disc_B = Disc_B.cuda()
    criterion_GAN.cuda()
    criterion_cycle.cuda()
    criterion_identity.cuda()

# Initialize weights
Gen_AB.apply(initialize_conv_weights_normal)
Gen_BA.apply(initialize_conv_weights_normal)
Disc_A.apply(initialize_conv_weights_normal)
Disc_B.apply(initialize_conv_weights_normal)

# Buffers for previously generated samples
fake_A_buffer = ReplayBuffer()
fake_B_buffer = ReplayBuffer()

# Optimizers
optimizer_G = torch.optim.Adam(
    itertools.chain(Gen_AB.parameters(), Gen_BA.parameters()),
    lr=hp.lr,
    betas=(hp.b1, hp.b2),
)
optimizer_Disc_A = torch.optim.Adam(Disc_A.parameters(), lr=hp.lr, betas=(hp.b1, hp.b2))
optimizer_Disc_B = torch.optim.Adam(Disc_B.parameters(), lr=hp.lr, betas=(hp.b1, hp.b2))

# Learning rate schedulers
lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(
    optimizer_G, lr_lambda=LambdaLR(hp.n_epochs, hp.epoch, hp.decay_start_epoch).step
)
lr_scheduler_Disc_A = torch.optim.lr_scheduler.LambdaLR(
    optimizer_Disc_A, lr_lambda=LambdaLR(hp.n_epochs, hp.epoch, hp.decay_start_epoch).step
)
lr_scheduler_Disc_B = torch.optim.lr_scheduler.LambdaLR(
    optimizer_Disc_B, lr_lambda=LambdaLR(hp.n_epochs, hp.epoch, hp.decay_start_epoch).step
)

# Training function
def train(
    Gen_BA,
    Gen_AB,
    Disc_A,
    Disc_B,
    train_dataloader,
    n_epochs,
    criterion_identity, 
    criterion_cycle,
    lambda_cyc,
    criterion_GAN,
    optimizer_G,
    fake_A_buffer,
    fake_B_buffer,
    clear_output,
    optimizer_Disc_A,
    optimizer_Disc_B,
    Tensor,
    sample_interval,
    lambda_id,
):
    prev_time = time.time()
    for epoch in range(hp.epoch, n_epochs):
        for i, batch in enumerate(train_dataloader):
            real_A = batch["A"].type(Tensor)
            real_B = batch["B"].type(Tensor)

            valid = torch.ones((real_A.size(0), *Disc_A.output_shape), requires_grad=False).type(Tensor)
            fake = torch.zeros((real_A.size(0), *Disc_A.output_shape), requires_grad=False).type(Tensor)

            Gen_AB.train()  # Horses are converted to Zebras(Fake)
            Gen_BA.train()  # Zebras are converted to Horses(Fake)

            optimizer_G.zero_grad()

            # Identity Loss 
            # First pass real A images to the Generator, that will generate A-domain images
            loss_id_A = criterion_identity(Gen_BA(real_A), real_A)
            # Then pass real B images to the Generator, that will generate B-domain images
            loss_id_B = criterion_identity(Gen_AB(real_B), real_B)
            loss_identity = (loss_id_A + loss_id_B) / 2

            # GAN losses for GAN_AB
            fake_B = Gen_AB(real_A)   # Gen_AB is generating fake B from real A images (horse to zebra)
            loss_GAN_AB = criterion_GAN(Disc_B(fake_B), valid)
            
            # GAN losses for GAN_BA  
            fake_A = Gen_BA(real_B)  # Gen_BA is generating fake A from real B images (zebra to horse)
            loss_GAN_BA = criterion_GAN(Disc_A(fake_A), valid)
            loss_GAN = (loss_GAN_AB + loss_GAN_BA) / 2
            
            # Cycle Consistency Loss
            reconstructed_A = Gen_BA(fake_B) # Here the fake zebra is being converted back to horse using Gen_BA
            loss_cycle_A = criterion_cycle(reconstructed_A, real_A) # Forward loss
            reconstructed_B = Gen_AB(fake_A) # Here the fake horse is being converted back zebra using Gen_AB
            loss_cycle_B = criterion_cycle(reconstructed_B, real_B) # Backward loss
            loss_cycle = (loss_cycle_A + loss_cycle_B) / 2

            loss_G = loss_GAN + lambda_cyc * loss_cycle + lambda_id * loss_identity
            loss_G.backward()
            optimizer_G.step()

            optimizer_Disc_A.zero_grad()
            loss_real = criterion_GAN(Disc_A(real_A), valid)
            fake_A_ = fake_A_buffer.push_and_pop(fake_A)
            loss_fake = criterion_GAN(Disc_A(fake_A_.detach()), fake)
            loss_Disc_A = (loss_real + loss_fake) / 2
            loss_Disc_A.backward()
            optimizer_Disc_A.step()

            optimizer_Disc_B.zero_grad()
            loss_real = criterion_GAN(Disc_B(real_B), valid)
            fake_B_ = fake_B_buffer.push_and_pop(fake_B)
            loss_fake = criterion_GAN(Disc_B(fake_B_.detach()), fake)
            loss_Disc_B = (loss_real + loss_fake) / 2
            loss_Disc_B.backward()
            optimizer_Disc_B.step()

            loss_D = (loss_Disc_A + loss_Disc_B) / 2

            batches_done = epoch * len(train_dataloader) + i
            batches_left = n_epochs * len(train_dataloader) - batches_done
            time_left = datetime.timedelta(
                seconds=batches_left * (time.time() - prev_time)
            )
            prev_time = time.time()

            print(
                "\r[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f, adv: %f, cycle: %f, identity: %f] ETA: %s"
                % (
                    epoch,
                    n_epochs,
                    i,
                    len(train_dataloader),
                    loss_D.item(),
                    loss_G.item(),
                    loss_GAN.item(),
                    loss_cycle.item(),
                    loss_identity.item(),
                    time_left,
                )
            )

            if batches_done % sample_interval == 0:
                clear_output()
                plot_output(save_img_samples(batches_done), 30, 40)

# Execute the final training function
train(
    Gen_BA=Gen_BA,
    Gen_AB=Gen_AB,
    Disc_A=Disc_A,
    Disc_B=Disc_B,
    train_dataloader=train_dataloader,
    n_epochs=hp.n_epochs,
    criterion_identity=criterion_identity,
    criterion_cycle=criterion_cycle,
    lambda_cyc=hp.lambda_cyc,
    criterion_GAN=criterion_GAN,
    optimizer_G=optimizer_G,
    fake_A_buffer=fake_A_buffer,
    fake_B_buffer=fake_B_buffer,
    clear_output=clear_output,
    optimizer_Disc_A=optimizer_Disc_A,
    optimizer_Disc_B=optimizer_Disc_B,
    Tensor=Tensor,
    sample_interval=hp.sample_interval,
    lambda_id=hp.lambda_id,
)


Not using CUDA


ValueError: num_samples should be a positive integer value, but got num_samples=0